# Databases - week 11
install package mysql-connector-python

In [ ]:
! pip install mysql-connector-python

## Connect to the database

In [ ]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()

print(dbconnection.database)
# do some stuff
cursor.close()
dbconnection.close()

movies_db


## CREATE OPERATION

In [3]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
query = "insert into actors(name) values('Keanu Reeves');"
cursor.execute(query)
dbconnection.commit()

cursor.close()
dbconnection.close()

In [4]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
query = "insert into actors(name) values(%s);"

data = [["Brandon Fraser"], ["Jean-Clause Van Damme"], ["Leonardo Di Caprio"], ["Dwayne 'the rock' Johnson"],
        ["Margot Robbie"], ["Brad Pitt"], ["Angelina Jolie"]]

cursor.executemany(query, data)

query = "insert into movies(title, synopsis) values(%s, %s);"
data = ("The Matrix", "Computer hacker discovers that everything is fake, joins the fight against the rebels and discovers his role in the fate of the war against controllers")

cursor.execute(query, data)

movies = [("Interstellar", "Earth is doomed, need to find new planet, tries a lot of failed discoveries, ends up discovering laws of time and space"),
          ("Jurassic Park", "Weird scientists close dinosaur dna and make a amusement park out of it that turns south quickly."),
          ("John Wick", "Super assassin loses his dog and goes bonkers."),
          ("Halloween", "Michael Meyers tries to make Halloween a fun time by killing random strangers. Lots of blood and scare")]

#insert each and every movie
for movie in movies:
    cursor.execute(query, movie)

dbconnection.commit()

cursor.close()
dbconnection.close()

## READ OPERATION

In [9]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
query = "select * from actors;"
cursor.execute(query)

data = cursor.fetchall()
print(data)

#also possible to print in different ways through your data
for actor in data: #everything
    print(actor)

for actor in data:
    print(actor[1]) #only names

for id, name in data:
    print(id, "+", name)

cursor.close()
dbconnection.close()


[(1, 'Michelle Yeoh'), (2, 'Stephan James'), (3, 'Jamie Lee Curtis'), (4, 'Tom Cruise'), (5, 'Keanu Reeves'), (6, 'Brandon Fraser'), (7, 'Jean-Clause Van Damme'), (8, 'Leonardo Di Caprio'), (9, "Dwayne 'the rock' Johnson"), (10, 'Margot Robbie'), (11, 'Brad Pitt'), (12, 'Angelina Jolie')]
(1, 'Michelle Yeoh')
(2, 'Stephan James')
(3, 'Jamie Lee Curtis')
(4, 'Tom Cruise')
(5, 'Keanu Reeves')
(6, 'Brandon Fraser')
(7, 'Jean-Clause Van Damme')
(8, 'Leonardo Di Caprio')
(9, "Dwayne 'the rock' Johnson")
(10, 'Margot Robbie')
(11, 'Brad Pitt')
(12, 'Angelina Jolie')
Michelle Yeoh
Stephan James
Jamie Lee Curtis
Tom Cruise
Keanu Reeves
Brandon Fraser
Jean-Clause Van Damme
Leonardo Di Caprio
Dwayne 'the rock' Johnson
Margot Robbie
Brad Pitt
Angelina Jolie
1 + Michelle Yeoh
2 + Stephan James
3 + Jamie Lee Curtis
4 + Tom Cruise
5 + Keanu Reeves
6 + Brandon Fraser
7 + Jean-Clause Van Damme
8 + Leonardo Di Caprio
9 + Dwayne 'the rock' Johnson
10 + Margot Robbie
11 + Brad Pitt
12 + Angelina Jolie


In [13]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()

query = """
        select movies.title, actors.name from movies
        inner join movie_actor on movies.id = movie_actor.movie_id
        inner join actors on actors.id = movie_actor.actor_id;
"""

cursor.execute(query)
data = cursor.fetchall()

print(data)

for row in data:
    print(row)

for movie, actor in data:
    print(actor, "played in", movie)

cursor.close()
dbconnection.close()

[('Everything Everywhere All at Once', 'Michelle Yeoh'), ('Everything Everywhere All at Once', 'Stephan James'), ('Everything Everywhere All at Once', 'Jamie Lee Curtis'), ('Top Gun: Maverick', 'Tom Cruise')]
('Everything Everywhere All at Once', 'Michelle Yeoh')
('Everything Everywhere All at Once', 'Stephan James')
('Everything Everywhere All at Once', 'Jamie Lee Curtis')
('Top Gun: Maverick', 'Tom Cruise')
Michelle Yeoh played in Everything Everywhere All at Once
Stephan James played in Everything Everywhere All at Once
Jamie Lee Curtis played in Everything Everywhere All at Once
Tom Cruise played in Top Gun: Maverick


## UPDATE OPERATION

In [ ]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()

#cursor.execute("insert into movies(title) values('Barbie')")
query = """
        update movies set synopsis ='Girls standing up against the Kens of the world'
        where title like 'Barbie'
"""

cursor.execute(query)

dbconnection.commit()
cursor.close()
dbconnection.close()


## SCRAPE DATA INTO DATABASE
https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films

In [ ]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

url = "https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films"
response = requests.get(url)
soup = BeautifulSoup(response.text)

#opening the db
dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()

table = soup.find("table")
rows = table.find_all("tr")
for row in rows:
    td = row.find_all("td")
    if len(td) > 0:
        title = td[0].get_text().strip().replace("'","")
        year = td[1].get_text().strip()
        awards = td[2].get_text().strip()
        nominations = td[3].get_text().strip()
    
        #check for duplicates
        query = f"select * from movies where title like '{title}'"
        print(query)
        cursor.execute(query)
        
        data = cursor.fetchall()
        if len(data) == 0:
            query = f"insert into movies(title) values('{title}')"
            cursor.execute(query)
            dbconnection.commit()

            movie_id = cursor.lastrowid

            query = f"insert into awards(awardshow_id, year, movie_id, awards, nominations) values('1', {year}, {movie_id}, {awards}, {nominations})"

            #values = (1, {year}, {movie_id}, {awards}, {nominations})
            cursor.execute(query)
            dbconnection.commit()

cursor.close()
dbconnection.close()
